# Usando os truques de treinamento de redes neurais!

In [1]:
%matplotlib inline
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(42)

mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

In [2]:
# GAMBIARRA: https://stackoverflow.com/questions/69687794/unable-to-manually-load-cifar10-dataset
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
import tensorflow as tf
from tensorflow import keras

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2022-11-07 12:54:04.043088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:54:04.814001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:54:04.835084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:54:04.835380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

True

In [5]:
# Lê o dataset CIFAR-10, https://keras.io/api/datasets/cifar10/
train_data, test_data = keras.datasets.cifar10.load_data()
X_train_full, y_train_full = train_data
X_test, y_test = test_data

Fonte: https://www.cs.toronto.edu/~kriz/cifar.html

<img src='cifar.png'></img>

O conjunto de treinamento tem $50000$ imagens de $32 \times 32$ pixels, com $3$ canais de cor. Portanto, o *array* `X_train_full` contendo esses dados tem formato $(\text{amostras}, \text{linhas}, \text{colunas}, \text{canais}) = (50000, 32, 32, 3)$

In [6]:
print(X_train_full.shape)

(50000, 32, 32, 3)


Cada canal de cor é representado como um inteiro entre $0$ e $255$, e isso cabe em um único *byte*. Para economizar espaço, o tipo de dados do *array* então será `uint8`, significando "inteiro sem-sinal de 8 bits". Ou para o povo do C/C++, o famoso `unsigned char`.

In [7]:
print(X_train_full.dtype)

uint8


A variável dependente `y_train_full` tem $50000$ amostras e apenas um valor por amostra. Para se manter consistente com os formatos de dados do *framework* Tensorflow, a variável dependente será representada por uma matriz-coluna de tamanho $(50000, 1)$ ao invés de um array simples de tamanho $(50000,)$

In [8]:
print(y_train_full.shape)

(50000, 1)


Como as classes são inteiros entre $0$ e $9$, um `uint8` basta para armazená-las

In [9]:
print(y_train_full.dtype)

uint8


Já o conjunto de testes tem $10000$ amostras, observe os tamanhos e tipos de dado de `X_test` e `y_test`:

In [10]:
print(X_test.shape, X_test.dtype)
print(y_test.shape, y_test.dtype)

(10000, 32, 32, 3) uint8
(10000, 1) uint8


O conjunto de teste, como de costume, é inviolável. Vamos dividir o conjunto de treinamento mais uma vez em duas partes: um conjunto que realmente será usado para treinamento, e outro para validação (que é o conjunto de testes de dentro do conjunto de treinamento pleno, e então não é inviolável).

Vamos separar $5000$ amostras para validação.

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full,
    y_train_full,
    test_size=5000,
)

print("X_train", X_train.shape, X_train.dtype)
print("y_train", y_train.shape, y_train.dtype)
print("X_valid", X_valid.shape, X_valid.dtype)
print("y_valid", y_valid.shape, y_valid.dtype)

X_train (45000, 32, 32, 3) uint8
y_train (45000, 1) uint8
X_valid (5000, 32, 32, 3) uint8
y_valid (5000, 1) uint8


Antes de mais nada, vamos criar *baselines* de desempenho para comparação: o classificador trivial e a Random Forest.

O classificador mais simples, mais trivial possível, é simplesmente dizer que toda amostra vem da mesma classe: a classe com maior representação no conjunto de treinamento. Vamos ver:

In [12]:
import pandas as pd

pd.Series(y_train.ravel()).value_counts(True).sort_values()

2    0.099400
3    0.099489
7    0.099800
5    0.099911
8    0.099933
9    0.100000
1    0.100289
4    0.100311
6    0.100333
0    0.100533
dtype: float64

Aparentemente todas as classes estão representadas de modo aproximadamente igual, sendo a maior proporção igual a $10.05 \%$. Então vamos adotar como classe "oficial do nosso classificador trivial a classe $0$. Agora vamos ver o desempenho no conjunto de validação:

In [13]:
pd.Series(y_valid.ravel()).value_counts(True).sort_values()

0    0.0952
6    0.0970
4    0.0972
1    0.0974
9    0.1000
8    0.1006
5    0.1008
7    0.1018
3    0.1046
2    0.1054
dtype: float64

Neste conjunto a classe $0$ tem, por acidente estatístico, a pior proporção! Paciência, a vida é assim! (Procure pelo fenômeno de "regressão para a média", muitas coisas da vida vão ficar mais claras...)

Portanto, o nosso desempenho do classificador trivial será uma acurácia de $9.5\%$.

Se você quiser uma estimativa melhor de desempenho do classificador trivial você pode fazer validação cruzada, ou mesmo estimar a estatística de acurácia usando *bootstrap* para obter uma distribuição do valor de acurácia. Mas nada disso será relevante: $9.5\%$ é muito baixo.

Vamos ver agora a RandomForest.

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [15]:
%%time
model = RandomForestClassifier(n_jobs=-1, random_state=42)
model.fit(X_train.reshape(X_train.shape[0], -1), y_train.ravel())

CPU times: user 13min 20s, sys: 1.82 s, total: 13min 22s
Wall time: 1min 13s


RandomForestClassifier(n_jobs=-1, random_state=42)

In [16]:
y_pred = model.predict(X_valid.reshape(X_valid.shape[0], -1))
print(accuracy_score(y_valid.ravel(), y_pred))

0.454


Ok, esse é o *score* a ser vencido! (Será? Você pode pensar em alguma ressalva aqui?)

Vamos agora construir uma rede neural das antigas: nada profunda, e com função de ativação sigmoide. Vamos usar um `batch_size` grande, pois os itens do dataset são pequenos, não precisamos de batches muito pequenos (o tamanho padrão é 32 no TensorFlow). Veremos o desempenho:

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.Dense(100, activation="sigmoid"))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "inicial")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

2022-11-07 12:55:49.435331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:55:49.436380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:55:49.437080: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:55:49.438376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:55:49.439314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Epoch 1/100
44/44 [==============================] - 1s 8ms/step - loss: 2.2512 - accuracy: 0.1647 - val_loss: 2.1847 - val_accuracy: 0.1976
Epoch 2/100
44/44 [==============================] - 0s 5ms/step - loss: 2.1565 - accuracy: 0.2160 - val_loss: 2.1536 - val_accuracy: 0.2022
Epoch 3/100
44/44 [==============================] - 0s 5ms/step - loss: 2.1125 - accuracy: 0.2397 - val_loss: 2.1136 - val_accuracy: 0.2294
Epoch 4/100
44/44 [==============================] - 0s 5ms/step - loss: 2.0827 - accuracy: 0.2564 - val_loss: 2.0993 - val_accuracy: 0.2596
Epoch 5/100
44/44 [==============================] - 0s 5ms/step - loss: 2.0643 - accuracy: 0.2615 - val_loss: 2.0488 - val_accuracy: 0.2824
Epoch 6/100
44/44 [==============================] - 0s 5ms/step - loss: 2.0391 - accuracy: 0.2738 - val_loss: 2.0408 - val_accuracy: 0.2848
Epoch 7/100
44/44 [==============================] - 0s 5ms/step - loss: 2.0248 - accuracy: 0.2810 - val_loss: 2.0336 - val_accuracy: 0.2668
Epoch 8/100
4

[1.7829521894454956, 0.3725999891757965]

Vamos partir para uma rede profunda agora. Vamos criar $10$ camadas *fully-connected*, função de ativação "elu", inicialização de pesos adequada para a respectiva função de ativação, e otimizador "adam". Vamos ver se melhora:

In [18]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "profunda")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 1s 11ms/step - loss: 99.2737 - accuracy: 0.1065 - val_loss: 14.1032 - val_accuracy: 0.1104
Epoch 2/100
44/44 [==============================] - 0s 7ms/step - loss: 6.3473 - accuracy: 0.1294 - val_loss: 3.5663 - val_accuracy: 0.1492
Epoch 3/100
44/44 [==============================] - 0s 6ms/step - loss: 2.9020 - accuracy: 0.1642 - val_loss: 2.6709 - val_accuracy: 0.1814
Epoch 4/100
44/44 [==============================] - 0s 7ms/step - loss: 4.3773 - accuracy: 0.1488 - val_loss: 3.3640 - val_accuracy: 0.1750
Epoch 5/100
44/44 [==============================] - 0s 7ms/step - loss: 2.6131 - accuracy: 0.1837 - val_loss: 2.3364 - val_accuracy: 0.2006
Epoch 6/100
44/44 [==============================] - 0s 7ms/step - loss: 2.2807 - accuracy: 0.2163 - val_loss: 2.2280 - val_accuracy: 0.2192
Epoch 7/100
44/44 [==============================] - 0s 7ms/step - loss: 2.2262 - accuracy: 0.2304 - val_loss: 2.2160 - val_accuracy: 0.2242
Epoch 8/10

[1.6967824697494507, 0.4004000127315521]

Muito melhor! Agora acertamos em torno de $43\%$ das classes!

A técnica de "batch normalization" pode melhorar ainda mais a convergência do modelo, vamos experimentar:

In [19]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

run_logdir = os.path.join(os.curdir, "cifar10_logs", "batchnorm")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [tensorboard_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 2s 19ms/step - loss: 1.8837 - accuracy: 0.3420 - val_loss: 4.2296 - val_accuracy: 0.1698
Epoch 2/100
44/44 [==============================] - 1s 12ms/step - loss: 1.5573 - accuracy: 0.4459 - val_loss: 2.2803 - val_accuracy: 0.2918
Epoch 3/100
44/44 [==============================] - 1s 12ms/step - loss: 1.4408 - accuracy: 0.4885 - val_loss: 1.8020 - val_accuracy: 0.3864
Epoch 4/100
44/44 [==============================] - 1s 12ms/step - loss: 1.3568 - accuracy: 0.5168 - val_loss: 1.6243 - val_accuracy: 0.4352
Epoch 5/100
44/44 [==============================] - 1s 12ms/step - loss: 1.2841 - accuracy: 0.5412 - val_loss: 1.5157 - val_accuracy: 0.4686
Epoch 6/100
44/44 [==============================] - 1s 12ms/step - loss: 1.2248 - accuracy: 0.5643 - val_loss: 1.4780 - val_accuracy: 0.4822
Epoch 7/100
44/44 [==============================] - 1s 12ms/step - loss: 1.1713 - accuracy: 0.5847 - val_loss: 1.4887 - val_accuracy: 0.4786
Epoch 

[4.852081775665283, 0.46459999680519104]

Uau. Isso sim que é *overfitting*! Para resolver o problema do *overfitting* vamos adotar o EarlyStopping:

In [20]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "earlystopping")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 2s 19ms/step - loss: 1.8856 - accuracy: 0.3408 - val_loss: 4.0880 - val_accuracy: 0.1616
Epoch 2/100
44/44 [==============================] - 1s 12ms/step - loss: 1.5500 - accuracy: 0.4481 - val_loss: 2.2514 - val_accuracy: 0.3068
Epoch 3/100
44/44 [==============================] - 1s 12ms/step - loss: 1.4429 - accuracy: 0.4864 - val_loss: 1.7794 - val_accuracy: 0.3974
Epoch 4/100
44/44 [==============================] - 1s 12ms/step - loss: 1.3543 - accuracy: 0.5172 - val_loss: 1.6120 - val_accuracy: 0.4414
Epoch 5/100
44/44 [==============================] - 1s 12ms/step - loss: 1.2889 - accuracy: 0.5393 - val_loss: 1.5482 - val_accuracy: 0.4624
Epoch 6/100
44/44 [==============================] - 1s 12ms/step - loss: 1.2257 - accuracy: 0.5673 - val_loss: 1.5137 - val_accuracy: 0.4650
Epoch 7/100
44/44 [==============================] - 1s 12ms/step - loss: 1.1751 - accuracy: 0.5822 - val_loss: 1.4961 - val_accuracy: 0.4778
Epoch 

[1.737622857093811, 0.487199991941452]

Melhorou mais! Mas ainda temos o problema do *overfitting*, vamos tentar a técnica do "drop-out" para ver se melhora:

In [21]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "dropout")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 3s 25ms/step - loss: 2.1503 - accuracy: 0.2419 - val_loss: 4.4301 - val_accuracy: 0.1742
Epoch 2/100
44/44 [==============================] - 1s 14ms/step - loss: 1.8289 - accuracy: 0.3386 - val_loss: 2.7147 - val_accuracy: 0.2922
Epoch 3/100
44/44 [==============================] - 1s 15ms/step - loss: 1.7336 - accuracy: 0.3732 - val_loss: 2.0117 - val_accuracy: 0.3726
Epoch 4/100
44/44 [==============================] - 1s 13ms/step - loss: 1.6731 - accuracy: 0.4027 - val_loss: 1.7061 - val_accuracy: 0.4238
Epoch 5/100
44/44 [==============================] - 1s 15ms/step - loss: 1.6265 - accuracy: 0.4177 - val_loss: 1.6193 - val_accuracy: 0.4456
Epoch 6/100
44/44 [==============================] - 1s 14ms/step - loss: 1.5854 - accuracy: 0.4341 - val_loss: 1.5505 - val_accuracy: 0.4486
Epoch 7/100
44/44 [==============================] - 1s 13ms/step - loss: 1.5527 - accuracy: 0.4458 - val_loss: 1.5174 - val_accuracy: 0.4702
Epoch 

[1.3330496549606323, 0.5338000059127808]

Melhorou ainda mais! E agora não tem mais tanto overfitting! Já que é assim, vamos aumentar ainda mais a rede e ver se melhora: vamos colocar $20$ camadas!

In [22]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "mais_profunda")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 8s 37ms/step - loss: 2.4549 - accuracy: 0.1464 - val_loss: 4.9615 - val_accuracy: 0.0718
Epoch 2/100
44/44 [==============================] - 1s 22ms/step - loss: 2.0776 - accuracy: 0.2250 - val_loss: 3.3804 - val_accuracy: 0.1842
Epoch 3/100
44/44 [==============================] - 1s 23ms/step - loss: 1.9548 - accuracy: 0.2723 - val_loss: 2.7722 - val_accuracy: 0.2662
Epoch 4/100
44/44 [==============================] - 1s 22ms/step - loss: 1.8773 - accuracy: 0.3101 - val_loss: 2.4118 - val_accuracy: 0.2964
Epoch 5/100
44/44 [==============================] - 1s 22ms/step - loss: 1.8304 - accuracy: 0.3307 - val_loss: 2.0647 - val_accuracy: 0.3402
Epoch 6/100
44/44 [==============================] - 1s 22ms/step - loss: 1.7851 - accuracy: 0.3541 - val_loss: 1.8998 - val_accuracy: 0.3724
Epoch 7/100
44/44 [==============================] - 1s 23ms/step - loss: 1.7407 - accuracy: 0.3723 - val_loss: 1.7897 - val_accuracy: 0.3980
Epoch 

[1.3698253631591797, 0.5171999931335449]

Parece que não melhorou muito, estava bom do jeito anterior. Para terminar, vamos usar um escalonador de taxa de aprendizado para melhorar a velocidade de convergência do nosso modelo.

In [23]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)
run_logdir = os.path.join(os.curdir, "cifar10_logs", "escalonador")

# Criação do modelo.
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(10):
    model.add(keras.layers.Dropout(rate=0.1))
    model.add(keras.layers.BatchNormalization())
    model.add(
        keras.layers.Dense(
            100,
            kernel_initializer="he_normal",
            activation="elu",
            use_bias=False,
        ))
model.add(
    keras.layers.Dense(
        10,
        kernel_initializer="glorot_normal",
        activation="softmax",
    ))

s = 90 * len(X_train) // 1024  # number of steps in 90 epochs (batch size 1024)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
)

# Otimização do modelo.
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
callbacks = [tensorboard_cb, early_stopping_cb]

model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=1024,
    validation_data=(X_valid, y_valid),
    callbacks=callbacks,
)

# Avaliação do modelo.
model.evaluate(X_valid, y_valid)

Epoch 1/100
44/44 [==============================] - 3s 21ms/step - loss: 2.0223 - accuracy: 0.2748 - val_loss: 57.8945 - val_accuracy: 0.1260
Epoch 2/100
44/44 [==============================] - 1s 14ms/step - loss: 1.7661 - accuracy: 0.3574 - val_loss: 5.3493 - val_accuracy: 0.2896
Epoch 3/100
44/44 [==============================] - 1s 14ms/step - loss: 1.6840 - accuracy: 0.3940 - val_loss: 2.4171 - val_accuracy: 0.3806
Epoch 4/100
44/44 [==============================] - 1s 14ms/step - loss: 1.6075 - accuracy: 0.4217 - val_loss: 1.7527 - val_accuracy: 0.4320
Epoch 5/100
44/44 [==============================] - 1s 14ms/step - loss: 1.5599 - accuracy: 0.4425 - val_loss: 1.6990 - val_accuracy: 0.4382
Epoch 6/100
44/44 [==============================] - 1s 14ms/step - loss: 1.5210 - accuracy: 0.4574 - val_loss: 1.5852 - val_accuracy: 0.4622
Epoch 7/100
44/44 [==============================] - 1s 14ms/step - loss: 1.4851 - accuracy: 0.4712 - val_loss: 1.5000 - val_accuracy: 0.4720
Epoch

[1.3703436851501465, 0.5252000093460083]

In [24]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.3327 - accuracy: 0.5397


[1.3326767683029175, 0.5396999716758728]

**Atividade**: Refaça a atividade da aula anterior (Fashion MNIST) mas tente usar os truques de melhoramento de desempenho das redes neurais vistos aqui nesta aula.